<a href="https://colab.research.google.com/github/rtolps/TensorflowCats2DogsUGATIT/blob/master/WorkingAnimetensorflowproper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UGATIT Tutorial

### Install stuff

Uninstall new tensor flow and install old one

In [ ]:
!pip uninstall -y tensorflow
!pip install opencv-python matplotlib numpy Pillow Flask imutils tensorflow-gpu==1.14.0 awscli

In [ ]:
from platform import python_version

print(python_version())

In [ ]:
!pip install git+https://github.com/onnx/tensorflow-onnx

In [ ]:
!git clone https://github.com/rtolps/TensorflowCats2DogsUGATIT
%cd TensorflowCats2DogsUGATIT

#(Optional) install Bazel

In [ ]:
BAZEL_VERSION = '0.20.0'

In [ ]:
!wget https://github.com/bazelbuild/bazel/releases/download/{BAZEL_VERSION}/bazel-{BAZEL_VERSION}-installer-linux-x86_64.sh

In [ ]:
!chmod +x bazel-{BAZEL_VERSION}-installer-linux-x86_64.sh

In [ ]:
!./bazel-{BAZEL_VERSION}-installer-linux-x86_64.sh

In [ ]:
!bazel

# Import Other Stuff

Import Summary tool


In [ ]:
import tensorflow.tools.graph_transforms as graph_transforms

Import graph freezer

In [ ]:
from tensorflow.python.tools import freeze_graph

Mount google drive (if you want to put in pretrained model) and unzip checkpoint folder make sure containing file within checkpoint folder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip checkpoint.zip

# Train

In [ ]:
!python main.py --dataset cat2dog --phase train --light True --iteration 50 --epoch 2

#Test

In [ ]:
!python main.py --dataset cat2dog --phase test --light True

Convert Graph using code from the Github page of anime tensorflow in closed issues

# **Export** (I fixed all of the errors with the script, it outputs frozen graph!!)

In [ ]:
!mkdir graphyfreezy

In [12]:
!python freezeGraph.py --checkpoint_dir checkpoint/UGATIT_light_cat2dog_lsgan_4resblock_6dis_1_1_10_10_1000_sn_smoothing --output output

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

# Clean up frozen graph per [snapchat's requirements](https://lensstudio.snapchat.com/guides/machine-learning/ml-frameworks/export-from-tensorflow/) (this doesn't seem to work)

In [ ]:
!python cleanup.py

# This code seems to work better, at least it outputs a file. File is really big (40MB) and doesn't look that much different from unoptomized file and needs to go on a little diet to make it less than 10MB if that is possible using [this method](https://stackoverflow.com/questions/51957336/how-to-properly-reduce-the-size-of-a-tensorflow-savedmodel) or one of [these methods](https://www.programcreek.com/python/example/114582/tensorflow.tools.graph_transforms.TransformGraph). Also it can't have "minimum" nodes, they're not compatible with snapchat but when I used the pretrained model there are no minimum nodes in the graph so solution found? However I get an error saying transpose layer not supported. I don't know if that's even possible but I will try my best.

In [ ]:
# export optimized graph
from tensorflow.tools.graph_transforms import TransformGraph
 
frozen_graph_filename = 'output.pb'
input_node_names = ['OneShotIterator']
output_node_names = ['generator_B/Tanh']
optimized_graph_def_from_pb  = export_from_frozen_graph(frozen_graph_filename, input_node_names, output_node_names, output_filename='output_optimized.pb')

#Attempted code from stack overflow (this is very broken)

In [ ]:
from tensorflow.python.tools import freeze_graph
from tensorflow.tools.graph_transforms import TransformGraph

def get_graph_def_from_file(graph_filepath):
  with ops.Graph().as_default():
    with tf.gfile.GFile(graph_filepath, 'rb') as f:
      graph_def = tf.GraphDef()
      graph_def.ParseFromString(f.read())
      return graph_def

def optimize_graph(model_dir, graph_filename, transforms, output_node):
  input_names = []
  output_names = [output_node]
  if graph_filename is None:
    graph_def = get_graph_def_from_saved_model(model_dir)
  else:
    graph_def = get_graph_def_from_file(os.path.join(model_dir, 
         graph_filename))
  optimized_graph_def = TransformGraph(graph_def, input_names,      
      output_names, transforms)
  tf.train.write_graph(optimized_graph_def, logdir=model_dir, as_text=False, 
     name='optimized_model.pb')
  print('Graph optimized!')

  transforms = ['remove_nodes(op=Identity)', 'merge_duplicate_nodes',
 'strip_unused_nodes','fold_constants(ignore_errors=true)',
 'fold_batch_norms']

optimize_graph(saved_model_dir, "output.pb" , transforms, 'head/predictions/class_ids')

def convert_graph_def_to_saved_model(export_dir, graph_filepath):
  if tf.gfile.Exists(export_dir):
    tf.gfile.DeleteRecursively(export_dir)
  graph_def = get_graph_def_from_file(graph_filepath)
  with tf.Session(graph=tf.Graph()) as session:
    tf.import_graph_def(graph_def, name='')
    tf.saved_model.simple_save(
        session,
        export_dir,
        inputs={
            node.name: session.graph.get_tensor_by_name(
                '{}:0'.format(node.name))
            for node in graph_def.node if node.op=='Placeholder'},
        outputs={'class_ids': session.graph.get_tensor_by_name(
            'head/predictions/class_ids:0')}
    )
    print('Optimized graph converted to SavedModel!')

optimized_export_dir = os.path.join(export_dir, 'optimized')
optimized_filepath = os.path.join(saved_model_dir, 'optimized_model.pb')
convert_graph_def_to_saved_model(optimized_export_dir, optimized_filepath)